In [3]:
# import torch
# import torch.nn.functional as F
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime
from utils import *

import matplotlib.pyplot as plt
%matplotlib inline
# Download NLTK model data (you need to do this once)
nltk.download("book")

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/kenny/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/kenny/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     /Users/kenny/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/kenny/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     /Users/kenny/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     /Users/kenny/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package dependency_t

True

In [62]:
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

In [61]:
with open('../data/reddit-comments-2015-08.csv', 'r', newline='', encoding='utf-8') as f:
    # Initalize a reader object
    reader = csv.reader(f, skipinitialspace=True)
    # Skip the header row
    # next(reader)  
    # Split full comments into sentences  - [nltk.sent_tokenize(x[0].lower()) for x in reader] - for the paragraph x[0] from the csv file, make it lowercase and tokenize all sentence
    # For all pararaphs in the csv file. * operator unpacks the list into individual sentences, and creates a single iterable
    # sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in reader])
    sentences = itertools.chain(*[nltk.sent_tokenize(str(x).lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    # Replace all sentence x in sentences with the start token, sentence body, and text token"
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print (f"Parsed {len(sentences)} sentences.")

Parsed 68647 sentences.


In [63]:
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

In [64]:
tokenized_sentences = [[word for word in sentence if word not in {'[', ']', '(', ')'}] for sentence in tokenized_sentences]

In [65]:
# # List of lists
print(tokenized_sentences[10])

['SENTENCE_START', 'a', 'dishonest', 'seller', "isn\\'t", 'going', 'to', 'run', 'the', 'check', 'in', 'the', 'first', 'place', '.', 'SENTENCE_END']


In [66]:
b = {}

for i in range (len(tokenized_sentences)):
    for word1, word2, in zip(tokenized_sentences[i], tokenized_sentences[i][1:]):
        # Create a tuple
        bigram = (word1, word2)
        # Index into the dictionary, update it by one
        b[bigram] = b.get(bigram, 0) + 1


In [67]:
# Taking the first 20,000 entries as our vocabulary
sorted(b.items(), key = lambda kv: -kv[1])

[(('.', 'SENTENCE_END'), 47244),
 (('SENTENCE_START', '``'), 9738),
 (("''", 'SENTENCE_END'), 7573),
 (('SENTENCE_START', 'i'), 6466),
 (("'", 'SENTENCE_END'), 5956),
 ((',', 'and'), 5707),
 (('.', "''"), 4960),
 ((',', 'but'), 4942),
 (('*', '*'), 4696),
 (('of', 'the'), 4317),
 (('in', 'the'), 4082),
 (('?', 'SENTENCE_END'), 3919),
 (('.', "'"), 3412),
 (('it', "'s"), 3313),
 ((',', 'i'), 3176),
 (('if', 'you'), 2899),
 (('do', "n't"), 2748),
 (('&', 'gt'), 2573),
 (('gt', ';'), 2573),
 (('to', 'be'), 2485),
 (('http', ':'), 2354),
 (('i', "'m"), 2336),
 (('SENTENCE_START', 'it'), 2273),
 (('SENTENCE_START', 'the'), 2240),
 (('``', 'SENTENCE_END'), 2116),
 (('to', 'the'), 2106),
 (('on', 'the'), 2059),
 (('``', 'i'), 1996),
 (('SENTENCE_START', "'"), 1870),
 (('!', 'SENTENCE_END'), 1804),
 (('is', 'a'), 1693),
 (('SENTENCE_START', 'if'), 1634),
 (('https', ':'), 1629),
 ((',', 'the'), 1599),
 (('and', 'i'), 1568),
 (('you', 'can'), 1563),
 (('amp', ';'), 1535),
 (('for', 'the'), 1529

In [76]:
word_set = set(word for bigram, count in sorted(b.items(), key=lambda kv: -kv[1])[:30000] for word in bigram)
word_set

{'\\n\\ni\\',
 'any',
 'election',
 'weight',
 'extent',
 'exists',
 'complaint',
 'pretty',
 'altogether',
 'hung',
 'obsessed',
 'committed',
 'leaders',
 'peace',
 'suggests',
 "'my",
 'different',
 'form',
 'chicken',
 '//www.reddit.com/r/askscience/search',
 'anything',
 'returned',
 'region',
 'tired',
 'myself',
 'native',
 '82',
 'wont',
 'submitter',
 'locked',
 'has',
 'width',
 'bowl',
 'brother',
 'reason',
 'concluded',
 'research',
 'breaches',
 'dig',
 'involved',
 'yup',
 'finally',
 'hundreds',
 'chose',
 'happy',
 'frequency',
 'criticism',
 'as',
 'nope',
 'projects',
 'invest',
 'holding',
 'your',
 'gcxrep',
 'scenario',
 'must',
 'message=myreminders',
 'c3',
 '//www.reddit.com/r/havoc_bot/wiki/',
 'box',
 '//0fs.me/yis5stledr\\n\\nhttp',
 'raw',
 'plot',
 'interaction',
 'roughly',
 '20a',
 'likely',
 'mistakes',
 '8gb',
 'honor',
 'privilege',
 'sense',
 'appreciated',
 'medication',
 'jail',
 'sick',
 'representation',
 'sneak',
 'according',
 'supporting',
 'e

In [69]:
dim = len(word_set)
dim

5138

In [70]:
stoi = {s:i for i,s in enumerate(word_set)} # string to index
itos = {i:s for s, i in stoi.items()}
stoi


{'\\n\\ni\\': 0,
 'any': 1,
 'election': 2,
 'weight': 3,
 'extent': 4,
 'exists': 5,
 'complaint': 6,
 'pretty': 7,
 'altogether': 8,
 'hung': 9,
 'obsessed': 10,
 'committed': 11,
 'leaders': 12,
 'peace': 13,
 'suggests': 14,
 "'my": 15,
 'different': 16,
 'form': 17,
 'chicken': 18,
 '//www.reddit.com/r/askscience/search': 19,
 'anything': 20,
 'returned': 21,
 'region': 22,
 'tired': 23,
 'myself': 24,
 'native': 25,
 '82': 26,
 'wont': 27,
 'submitter': 28,
 'locked': 29,
 'has': 30,
 'width': 31,
 'bowl': 32,
 'brother': 33,
 'reason': 34,
 'concluded': 35,
 'research': 36,
 'breaches': 37,
 'dig': 38,
 'involved': 39,
 'yup': 40,
 'finally': 41,
 'hundreds': 42,
 'chose': 43,
 'happy': 44,
 'frequency': 45,
 'criticism': 46,
 'as': 47,
 'nope': 48,
 'projects': 49,
 'invest': 50,
 'holding': 51,
 'your': 52,
 'gcxrep': 53,
 'scenario': 54,
 'must': 55,
 'message=myreminders': 56,
 'c3': 57,
 '//www.reddit.com/r/havoc_bot/wiki/': 58,
 'box': 59,
 '//0fs.me/yis5stledr\\n\\nhttp':

In [71]:
found = False
for sentence in tokenized_sentences:
    if 'UNKNOWN_TOKEN' in sentence:
        found = True
        break

if found:
    print("'UNKNOWN_TOKEN' exists in the list tokenized_sentences.")
else:
    print("'UNKNOWN_TOKEN' does not exist in the list tokenized_sentences.")

'UNKNOWN_TOKEN' does not exist in the list tokenized_sentences.


In [112]:
tokenized_sentences[0]

['SENTENCE_START',
 'this',
 'is',
 'definitely',
 'where',
 'it\\',
 "'s",
 'at',
 '.',
 "'",
 'SENTENCE_END']

In [125]:
tokenized_words = [word for sentence in tokenized_sentences for word in sentence]


In [122]:
tokenized_words = [word for sentence in tokenized_sentences for word in sentence if word in word_set]


In [ ]:
tokenized_words = [ch for sentence in tokenized_sentences for word in sentence if word in word_set]


In [139]:
tokenized_words = [stoi[word] for word in tokenized_words]


In [127]:
print(len(tokenized_words))

1500138


In [141]:
import random
random.seed(42)
random.shuffle(tokenized_sentences)
n1 = int(0.8 * len(tokenized_sentences))
n2 = int(0.9 * len(tokenized_sentences))

Xtr = tokenized_words[:n1]
Ytr = tokenized_words[1:n1+1]
Xdev = tokenized_words[n1:n2]
Ydev = tokenized_words[n1+1:n2+1]

In [143]:
print(Xtr[:10])
print(Ytr[:10])

[773, 659, 850, 3789, 3610, 2416, 4409, 3824, 3684, 4608]
[659, 850, 3789, 3610, 2416, 4409, 3824, 3684, 4608, 179]


In [147]:
vocab_size = 5138
def lossFun(inputs, targets, hprev):
    """
    inputs, targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    # Create four dictionaries
    xs, hs, ys, ps = {}, {}, {}, {}
    # Copy the last hidden state into hprev
    hs[-1] = np.copy(hprev)
    loss = 0
    
    ### Forward pass (each time step)
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        # At this time step, input an encoded char, as a (27, 5) one hot vector
        xs[t][inputs[t]] = 1
        # At this time step, follow the formulas to get the hidden state at this time step
        # (100,27) @ (27, 1) = (100, 1), (100, 100) @ (100, 1) = (100, 1) + (100, 1) -> (100, 1) (column vector of 100 outputs
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state

        # At this time step, compute the output state
        # (27, 100) @ (100, 1) = (27, 1) + (27, 1) = (1, 27)
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars

        # Normalize our probabiltiies using softmax
        # ps = (1, 27)
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars

        # Take the loss of the correct output probabilities
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    ### Backward pass: compute gradients going backwards
    # dWxh (100, 27), dWhh (100, 100), dWhy = (27, 100)
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    # dbh (100, 1),  dby (27, 1)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    # (100, 1)
    # Start off with zero
    dhnext = np.zeros_like(hs[0]) 
    
    for t in reversed(range(len(inputs))):
        # Backpropogate through the softmax to the logits
        # (27, 5)
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here

        # (27, 5) @ (5, 100) = (27, 100)
        dWhy += np.dot(dy, hs[t].T)
        # (27, 5) -> This secrety should sum across the columns dim = 1 to produce (27, 1)
        dby += dy

        # dh(t), dh(t)raw
        # Backpropogate into the previous layer (100, 27) @ (27, 5) = (100, 5)
        dh = np.dot(Why.T, dy) + dhnext
        # Backprop through tanh nonlinearity (element-wise forward -> element-wise backward)
        dhraw = (1 - hs[t] * hs[t]) * dh 

        # dWxh, dWhh, dbh
        # dWxh = (100, 5) @ (5, 27) = (100, 27)
        dWxh += np.dot(dhraw, xs[t].T)
        # dWhh = (100, 5) (5, 100) = (100, 100)
        dWhh += np.dot(dhraw, hs[t-1].T)
        # Calculate dbh and dWxh
        dbh += dhraw

        # Derive part of the next time step's gradient
        dhnext = np.dot(Whh.T, dhraw)

    # Clipping gradients
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]


In [162]:
def sample(h, seed_ix, n): # Passed in the previous inputs, the first letter of input, and 200 samples
    """ 
    Sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    # Create a one hot vector
    x = np.zeros((vocab_size, 1))
    x[stoi['SENTENCE_START']] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes


In [163]:
# Hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# Model parameters
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 # input to hidden - (100, 27)
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 # hidden to hidden (100, 100)
Why = np.random.randn(vocab_size, hidden_size) * 0.01 # hidden to output (27, 100)
bh = np.zeros((hidden_size, 1)) # hidden bias (100, 1)
by = np.zeros((vocab_size, 1)) # output bias (27, 1)

n, p = 0, 0

mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)

mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad

smooth_loss = -np.log(1.0/vocab_size) * seq_length # loss at iteration 0

while True:
    # Prepare inputs (Sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # Go from start of data
    inputs = Xtr[p:p+seq_length]
    targets = Ytr[p:p+seq_length]

    
    # Sample from the model now and then, and print the result
    if n % 100 == 0:
        # Pass in the previous inputs, the first letter of input, and 200 samples
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ' '.join(itos[ix] for ix in sample_ix)
        print ('----\n %s \n----' % (txt, ))

    
    # Forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: 
        print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update   
    p += seq_length # move data pointer
    n += 1 # iteration counter 

----
 contacted articles bit however place shooters ndp ie picks individual weekends //www.google.com/search cast sciences \n\n6 individuals social drinks cpu results . competing viable air boring 400 raids susceptible question relay stuff \n\nwhat damage number to=remindmebot coast almost unit player belief relation involving alright pretty \n\nunfortunately pulled alcohol fingers front basic finally refused relay generate keyboard 'try affect copy s talked paid theory monster puts happy approach carrier step kidding spectrum mouth modern political santa hill \n\nunfortunately kit to=/r/music 5th walking ridiculous below ashley -\n\n\n while technical bothered bottom cycle unique ^^^suggestions still whatsoever application allows \nkarmadecay| yea gg clothes cried original edited mass band navy solution battles involved bit toward sufficient issue while theme ^^bot \n^^^ notes considered pills far hardware bomb neat 20eye mode average on\nhttp catching dollars worries together violati

KeyboardInterrupt: 

In [156]:
stoi['SENTENCE_END']

179

In [158]:
stoi['SENTENCE_START']

773

In [164]:
sample_ix = sample(hprev, 773, 200)
txt = ' '.join(itos[ix] for ix in sample_ix)
print ('----\n %s \n----' % (txt, ))

----
 inclined made me stuff much competing companies 'm off gt me stuff either competing obligation model off '' me stuff much 'most stranger 'm and '' '' stuff campus creates lying suitable off stacks me stuff obviously competing companies 'm off '' buy stuff knowing competing more 'm tired saiyan stuff stuff much competing i\ 'm inclined '' names expense much cs i+4 equation parties imagine buy stuff fucked competing whining arm description loop me stuff much mouse ^gee turkey inclined '' buy '' much competing companies attacks inclined cheers me /scrunchieshy-intensifies much competing i\ attacks inclined sell me stuff much competing lifestyle feeling inclined is buy stuff much competing more 'm timing steal me stuff much competing president months inclined \n\nyou me stuff target competing agents mid off beer buy mile much competing i\ 'm off '' me so just competing seeks 'm inclined is buy stuff them competing bodies attacks inclined '' stuff stuff much competing sadly settings o